In [1]:
import os,sys,tqdm
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *

from collections import Counter
import pandas as pd
import shutil
import pickle
import gc
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [3]:
max_words = 8352#8352 #Top most frequent words to consider. Any less frequent word will appear as oov_char value in the sequence data.
max_length = 360#360

In [4]:
word_index = reuters.get_word_index()
print('all_words#:',len(word_index))
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,maxlen=max_length,
                                                         test_split=0.2,seed=830913)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

all_words#: 30979
8260 train sequences
2066 test sequences


In [13]:
# train_len = [len(x) for x in x_train]
# test_len = [len(x) for x in x_test]
# all_len = train_len
# all_len.extend(test_len)
# basic_statistics(all_len)

,length
mean,145.964197
std,145.878476
mode,17.000000
min,2.000000
q1,60.000000
median,95.000000
q3,180.000000
max,2376.000000
iqr,120.000000
outlier,360.000000


In [15]:
# df = pd.DataFrame(all_len)
# df.to_excel('./results/length_dist.xlsx', header=False, index=False)
# df

In [21]:
# train_words = []
# for x in x_train:
#     train_words.extend(x)
# test_words = []
# for x in x_test:
#     test_words.extend(x)
# all_words = train_words
# all_words.extend(test_words)
# all_statistcs = Counter(all_words)
# all_statistcs

Counter({1: 11228,
         53: 4213,
         352: 647,
         26: 8451,
         14: 15015,
         279: 801,
         39: 5818,
         72: 3091,
         4497: 26,
         18: 11039,
         83: 2597,
         5291: 21,
         88: 2381,
         5397: 20,
         11: 20141,
         3412: 37,
         19: 10755,
         151: 1363,
         230: 962,
         831: 253,
         15: 13329,
         165: 1232,
         318: 707,
         3780: 33,
         124: 1676,
         1527: 117,
         1424: 128,
         35: 6588,
         5302: 20,
         12: 16668,
         17: 11191,
         486: 459,
         341: 663,
         142: 1466,
         255: 870,
         219: 997,
         429: 528,
         68: 3363,
         146: 1402,
         252: 882,
         191: 1098,
         15448: 3,
         3631: 35,
         2283: 65,
         71: 3120,
         10: 29581,
         342: 660,
         49: 4565,
         1977: 80,
         324: 695,
         27: 8311,
         9222: 

In [28]:
# df = pd.DataFrame.from_dict(dict(all_statistcs), orient = 'index')
# df.to_excel('./results/words_dist2.xlsx', header=False, index=True)
# df

,0
1,11228
53,4213
352,647
26,8451
14,15015
...,...
24452,1
18567,2
27222,1
26864,1


In [5]:
trainX = tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_length,padding='post',value=0)
testX = tf.keras.preprocessing.sequence.pad_sequences(x_test,maxlen=max_length,padding='post',value=0)
print(trainX.shape, testX.shape)

(8260, 360) (2066, 360)


In [14]:
hidden_dim = 256
do = 0.1

## Graph execution
### Embedder

In [42]:
int_id = Input(shape=(max_length,), dtype='int32', name='int_ids') # 輸入的api funvtion name ID
int_ids = Masking(mask_value=0)(int_id)
sent_emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')(int_ids) 

### Encoder

In [43]:
rnn = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(sent_emb)
rnn = BatchNormalization(name='bn')(rnn)


### Filter

In [44]:
fil = TimeDistributed(Dense(1,activation='sigmoid',
                             name='filter_out'),name='TD2')(rnn)

### Classfier

In [47]:
mul = Multiply()([fil,sent_emb])
clf = LSTM(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm')(mul)
clf = BatchNormalization(name='bn3')(clf)
clf = Dense(max(y_train)+1,activation='softmax',name='clf')(clf)

## Compile

In [48]:
model = Model(inputs=int_id, outputs = clf)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
int_ids (InputLayer)            [(None, 358)]        0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 358)          0           int_ids[0][0]                    
__________________________________________________________________________________________________
glove_emb (Embedding)           (None, 358, 128)     1280000     masking_1[0][0]                  
__________________________________________________________________________________________________
common_extract (GRU)            (None, 358, 64)      37248       glove_emb[0][0]                  
______________________________________________________________________________________________

In [53]:
# loss
import keras.backend as K
def custom_objective(layer):
    return K.sum(layer.output)
#     return K.sum(layer.output)
# kk = tf.keras.backend.ea
model.compile(loss=custom_objective(model.get_layer(name='TD2')),optimizer='adam')

Using TensorFlow backend.


OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

## Eager Execution

In [52]:
# whole model
do = 0
init = tensorflow.keras.initializers.Ones()
class base_model(Model):
    def __init__(self):
        super(base_model, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn1 = BatchNormalization(name='bn1')
        self.fil = Dense(1,activation='hard_sigmoid',kernel_initializer=init,bias_initializer=init,name='filter_out')
        #self.fil = TimeDistributed(Dense(1,activation='sigmoid', name='filter_out'),name='TD2')
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = LSTM(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')
    def transform(self,x):
        return tf.math.round(x)
    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.rnn1(x1)
        x = self.bn1(x)
        y = self.fil(x)
        y1 = self.transform(y)
        x2 = self.mul([y1,x1])
        x = self.rnn2(x2) #x
        x = self.bn2(x)
        y2 = self.out(x)
        return y,y1,y2
        #return y,y1,y2,x2
        
model = base_model()

In [364]:
# partial1 model
init_w = tensorflow.keras.initializers.Constant(value=0.8)
init_b = tensorflow.keras.initializers.Constant(value=0.4)
def onezero(x):
    portion = 1#0.6#0.6 #0.6~1
    z = tf.where(x>=1.0, x - x + 1.0, x)
    y = tf.where(z<=0.0, z - z + 0.0, portion*z)
    return y

class base_model_1(Model):
    def __init__(self):
        super(base_model_1, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.att = Attention(name='selfatt')
        self.bn1 = BatchNormalization(name='bn1')
        #self.fil = Dense(1,activation='sigmoid',name='filter_out')
        self.fil = TimeDistributed(Dense(1,activation=onezero,kernel_initializer=init_w,bias_initializer=init_b, name='filter_out'),name='TD2') #relu/linear/step function

    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.att([x1,x1])
        #x = self.rnn1(x1)
        #x = self.bn1(x)
        y = self.fil(x)
        return x1,y

model1 = base_model_1()

In [365]:
# partial2 model
class base_model_2(Model):
    def __init__(self):
        super(base_model_2, self).__init__()
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = GRU(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')

    def call(self,x1,y1):
        x2 = self.mul([y1,x1])
        x = self.rnn3(x2) #x2 #y1=weight|binary
        x = self.bn2(x)
        y2 = self.out(x)
        return y2
    
model2 = base_model_2()

In [448]:
# TEST
x = tf.random.uniform((32, 6))
out1,out2 = model1(x)
out2 = tf.squeeze(out2,axis=-1)
# out2 = out2.astype('float32')
# out2 = tf.dtypes.cast(out2, tf.float8)
# print(out2.shape)
yy = tf.matmul(out2,kk)
print(yy[0].shape)
tf.where(yy[0]==0,1,0)

(32, 4)


<tf.Tensor: id=5770753, shape=(32, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]], dtype=int32)>

In [450]:
gg = tf.range(-5,5)
gg = tf.expand_dims(gg,axis=0)
gg = tf.keras.backend.repeat_elements(gg,rep=32,axis=0)
tf.where(gg==1,gg,0)

<tf.Tensor: id=5770779, shape=(32, 10), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    

In [438]:
kk = tf.Variable(np.array([[1.0,1.0,1.0,0.0,0.0,0.0],[0.0,1.0,1.0,1.0,0.0,0.0],[0.0,0.0,1.0,1.0,1.0,0.0],
                           [0.0,0.0,0.0,1.0,1.0,1.0]]).T,dtype='float32')
kk = tf.expand_dims(kk,axis=0)
kk = tf.keras.backend.repeat_elements(kk,rep=1,axis=0)
kk.shape

TensorShape([1, 6, 4])

In [366]:
batch_size = 128 #,reshuffle_each_iteration=True
train_ds = tf.data.Dataset.from_tensor_slices((trainX,y_train)).shuffle(trainX.shape[0]).batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((testX,y_test)).batch(batch_size)

In [367]:
def loss_object1(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def one_percentage(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer1 = tf.keras.optimizers.Nadam()
optimizer2 = tf.keras.optimizers.RMSprop()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_ones = tf.keras.metrics.Mean(name='train_ones')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_ones = tf.keras.metrics.Mean(name='test_ones')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [368]:
# seperate partial model
alpha = 0.0 #pahse1: -0.1 / 0.0 ; phase2: 0.01~0.05~0.1
beta = 1.0
gamma = 0.0
seq_num = 4 #連續幾個才叫做連續
@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=False) as tape:
        emb, pred_imp = model1(x)
        #loss1 = alpha*loss_object1(pred_imp) #phase1
        #pred_imp2 = tf.math.round(pred_imp)
        #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
        pred_imp2 = tf.math.round(pred_imp)
        loss1 = alpha*(1-loss_object1(pred_imp)) #pahse2:alpha*loss_object1(pred_imp) ; phase1: alpha*(1-loss_object1(pred_imp))
        pred_cat = model2(emb,pred_imp2) #pahse1: pred_imp; phase2; pred_imp2
        loss2 = beta*loss_object2(yc, pred_cat)
        loss = loss1 + loss2
    trainable_variable = model1.trainable_variables
    trainable_variable.extend(model2.trainable_variables)
    gradients = tape.gradient(loss,trainable_variable)
    optimizer1.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss)
    train_accuracy(yc, pred_cat)
    ones = one_percentage(pred_imp2) #pred_imp2
    train_ones(ones)
    
@tf.function
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = alpha*(1-loss_object1(pred_imp)) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    #t_loss = loss1 + loss2
    t_loss = loss1 + loss2
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)

In [331]:
'''
#AIO
alpha = 0.1
beta = 1
gamma = 0
@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=True) as tape: #persistent=True
        pred_imp,pred_round , pred_cat = model(x)
#         pred_cat = model(x)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
        loss1 = alpha*loss_object1(pred_imp)
        loss2 = beta*loss_object2(yc,pred_cat)
        loss = loss_object2(yc,pred_cat)
#     gradients = tape.gradient(loss, model.trainable_variables)
    grad1 = tape.gradient(loss1, model.trainable_variables)
    grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer1.apply_gradients(zip(gradients, model.trainable_variables))
    optimizer1.apply_gradients(zip(grad1, model.trainable_variables))
    optimizer2.apply_gradients(zip(grad2, model.trainable_variables))
#     with tf.GradientTape() as tape:
#         pred_imp , pred_cat = model(x)
#         loss2 = loss_object2(yc,pred_cat)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
#     grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer2.apply_gradients(zip(grad2, model.trainable_variables))

    train_loss(loss)
    train_accuracy(yc, pred_cat)
    ones = one_percentage(pred_round)
    train_ones(ones)
    
@tf.function
def test_step(x,yc):
    pred_imp,pred_round, pred_cat = model(x)
#     pred_cat = model(x)
    t_loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat) 
#     t_loss = loss_object2(yc,pred_cat)
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_round)
    test_ones(t_ones)
'''    

In [369]:
EPOCHS = 5000
gc.collect()
for epoch in range(EPOCHS):
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        test_step(test_text, test_labels)

    template = 'Epoch {}, Loss: {},Ones#: {}, Accuracy: {}%, Test Loss: {},Test Ones#: {}, Test Accuracy: {}%'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_ones.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_ones.result(),
                        test_accuracy.result()*100))

    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_ones.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_ones.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 2.5381412506103516,Ones#: 0.9465202689170837, Accuracy: 37.154964447021484, Test Loss: 2.371966600418091,Test Ones#: 0.9449232816696167, Test Accuracy: 37.173282623291016
Epoch 2, Loss: 2.3246042728424072,Ones#: 0.9287824630737305, Accuracy: 38.305084228515625, Test Loss: 2.16237735748291,Test Ones#: 0.7554682493209839, Test Accuracy: 46.7086181640625
Epoch 3, Loss: 1.96442711353302,Ones#: 0.7594312429428101, Accuracy: 52.990318298339844, Test Loss: 1.8882560729980469,Test Ones#: 0.7408806681632996, Test Accuracy: 55.32429885864258
Epoch 4, Loss: 1.8791507482528687,Ones#: 0.7618997693061829, Accuracy: 55.774818420410156, Test Loss: 1.8698310852050781,Test Ones#: 0.7427176237106323, Test Accuracy: 55.95353317260742
Epoch 5, Loss: 1.834506630897522,Ones#: 0.7740329504013062, Accuracy: 56.416465759277344, Test Loss: 1.8537601232528687,Test Ones#: 0.7509351372718811, Test Accuracy: 56.14714813232422
Epoch 6, Loss: 1.779542088508606,Ones#: 0.7791115641593933, Accuracy: 56.961

Epoch 46, Loss: 0.26901775598526,Ones#: 0.8073880076408386, Accuracy: 91.34382629394531, Test Loss: 1.690752387046814,Test Ones#: 0.7860807180404663, Test Accuracy: 71.49080657958984
Epoch 47, Loss: 0.27858495712280273,Ones#: 0.8020925521850586, Accuracy: 91.27118682861328, Test Loss: 1.6834931373596191,Test Ones#: 0.7699441909790039, Test Accuracy: 70.61955261230469
Epoch 48, Loss: 0.2786586582660675,Ones#: 0.7983691096305847, Accuracy: 91.25907897949219, Test Loss: 1.6540191173553467,Test Ones#: 0.7722068428993225, Test Accuracy: 72.02323150634766
Epoch 49, Loss: 0.3399449288845062,Ones#: 0.7839993238449097, Accuracy: 89.52784729003906, Test Loss: 1.6630362272262573,Test Ones#: 0.7538724541664124, Test Accuracy: 70.9583740234375
Epoch 50, Loss: 0.3449055254459381,Ones#: 0.7765752077102661, Accuracy: 89.70944213867188, Test Loss: 1.7396776676177979,Test Ones#: 0.7449431419372559, Test Accuracy: 69.4578857421875
Epoch 51, Loss: 0.3850165009498596,Ones#: 0.7599529027938843, Accuracy: 88

Epoch 90, Loss: 0.782025933265686,Ones#: 0.003316839225590229, Accuracy: 78.68038940429688, Test Loss: 1.7869422435760498,Test Ones#: 0.00590022187680006, Test Accuracy: 66.55372619628906
Epoch 91, Loss: 0.7596284747123718,Ones#: 0.003722445573657751, Accuracy: 79.29782104492188, Test Loss: 1.7669309377670288,Test Ones#: 0.008691866882145405, Test Accuracy: 66.31171417236328
Epoch 92, Loss: 0.7434994578361511,Ones#: 0.00461634760722518, Accuracy: 79.8062973022461, Test Loss: 1.7947767972946167,Test Ones#: 0.005857561714947224, Test Accuracy: 67.32817077636719
Epoch 93, Loss: 0.7375726103782654,Ones#: 0.004112113732844591, Accuracy: 79.74576568603516, Test Loss: 1.763034701347351,Test Ones#: 0.004069134127348661, Test Accuracy: 66.26331329345703
Epoch 94, Loss: 0.7606036067008972,Ones#: 0.0027860102709382772, Accuracy: 79.28572082519531, Test Loss: 1.8012734651565552,Test Ones#: 0.003043811535462737, Test Accuracy: 66.02130126953125
Epoch 95, Loss: 0.7559478878974915,Ones#: 0.0027024280

Epoch 133, Loss: 0.5665998458862305,Ones#: 0.0012704780092462897, Accuracy: 83.26876831054688, Test Loss: 2.1428616046905518,Test Ones#: 0.001354074222035706, Test Accuracy: 65.10164642333984
Epoch 134, Loss: 0.5424597859382629,Ones#: 0.0013007111847400665, Accuracy: 83.89830780029297, Test Loss: 2.1788384914398193,Test Ones#: 0.0014342693611979485, Test Accuracy: 64.90803527832031
Epoch 135, Loss: 0.5400088429450989,Ones#: 0.0012780119432136416, Accuracy: 84.09201049804688, Test Loss: 2.1540465354919434,Test Ones#: 0.001285753445699811, Test Accuracy: 63.891578674316406
Epoch 136, Loss: 0.5437975525856018,Ones#: 0.0012648215051740408, Accuracy: 84.14043426513672, Test Loss: 2.1870670318603516,Test Ones#: 0.0013127783313393593, Test Accuracy: 64.71442413330078
Epoch 137, Loss: 0.5342485308647156,Ones#: 0.0012904159957543015, Accuracy: 84.55205535888672, Test Loss: 2.186157703399658,Test Ones#: 0.0013405181234702468, Test Accuracy: 63.504356384277344
Epoch 138, Loss: 0.5189542770385742,

Epoch 176, Loss: 0.3980101943016052,Ones#: 0.0012168324319645762, Accuracy: 87.83293151855469, Test Loss: 2.6338024139404297,Test Ones#: 0.001145126880146563, Test Accuracy: 62.923519134521484
Epoch 177, Loss: 0.3986278772354126,Ones#: 0.0011109536280855536, Accuracy: 87.6150131225586, Test Loss: 2.634897232055664,Test Ones#: 0.0011374513851478696, Test Accuracy: 63.45595169067383
Epoch 178, Loss: 0.3886961042881012,Ones#: 0.0010955771431326866, Accuracy: 87.74818420410156, Test Loss: 2.624540328979492,Test Ones#: 0.0011187659110873938, Test Accuracy: 63.891578674316406
Epoch 179, Loss: 0.3788645565509796,Ones#: 0.0010864045470952988, Accuracy: 88.19612884521484, Test Loss: 2.6946961879730225,Test Ones#: 0.001119588385336101, Test Accuracy: 62.43949508666992
Epoch 180, Loss: 0.3852732479572296,Ones#: 0.0010923728113994002, Accuracy: 87.84503173828125, Test Loss: 2.67610239982605,Test Ones#: 0.0011293529532849789, Test Accuracy: 63.35914611816406
Epoch 181, Loss: 0.3971914052963257,Ones

...

Epoch 4820, Loss: 0.0739351436495781,Ones#: 0.0008710829424671829, Accuracy: 95.53268432617188, Test Loss: 3.8664937019348145,Test Ones#: 0.0009333271300420165, Test Accuracy: 62.5847053527832
Epoch 4821, Loss: 0.07240399718284607,Ones#: 0.0008717775344848633, Accuracy: 95.50847625732422, Test Loss: 3.864964246749878,Test Ones#: 0.0009334199130535126, Test Accuracy: 62.53630065917969
Epoch 4822, Loss: 0.07266855239868164,Ones#: 0.0008705919026397169, Accuracy: 95.46005249023438, Test Loss: 3.871488571166992,Test Ones#: 0.0009334256174042821, Test Accuracy: 62.778316497802734
Epoch 4823, Loss: 0.07326333224773407,Ones#: 0.000870370480697602, Accuracy: 95.59322357177734, Test Loss: 3.873936891555786,Test Ones#: 0.0009333371999673545, Test Accuracy: 62.5847053527832
Epoch 4824, Loss: 0.0732874721288681,Ones#: 0.0008714285213500261, Accuracy: 95.617431640625, Test Loss: 3.894831895828247,Test Ones#: 0.0009352614288218319, Test Accuracy: 62.63311004638672
Epoch 4825, Loss: 0.074676424264907

Epoch 4863, Loss: 0.0899789109826088,Ones#: 0.0008535143570043147, Accuracy: 95.03632354736328, Test Loss: 3.8454809188842773,Test Ones#: 0.0009143853676505387, Test Accuracy: 62.05227279663086
Epoch 4864, Loss: 0.08444075286388397,Ones#: 0.0008483403362333775, Accuracy: 95.16948699951172, Test Loss: 3.8928756713867188,Test Ones#: 0.000915895274374634, Test Accuracy: 61.95547103881836
Epoch 4865, Loss: 0.08441273123025894,Ones#: 0.0008503076387569308, Accuracy: 95.24213409423828, Test Loss: 3.9107518196105957,Test Ones#: 0.0009168633259832859, Test Accuracy: 61.61665344238281
Epoch 4866, Loss: 0.07939408719539642,Ones#: 0.000852162775117904, Accuracy: 95.23002624511719, Test Loss: 3.914055824279785,Test Ones#: 0.0009247309062629938, Test Accuracy: 61.47144317626953
Epoch 4867, Loss: 0.08295657485723495,Ones#: 0.0008599149296060205, Accuracy: 95.26634216308594, Test Loss: 3.8583054542541504,Test Ones#: 0.0009248413261957467, Test Accuracy: 61.6650505065918
Epoch 4868, Loss: 0.0815573409

Epoch 4906, Loss: 0.09609832614660263,Ones#: 0.000869544455781579, Accuracy: 95.10895538330078, Test Loss: 3.78411602973938,Test Ones#: 0.0009315609931945801, Test Accuracy: 62.003875732421875
Epoch 4907, Loss: 0.08725722879171371,Ones#: 0.0008645573980174959, Accuracy: 95.09684753417969, Test Loss: 3.846583604812622,Test Ones#: 0.0009282444952987134, Test Accuracy: 61.95547103881836
Epoch 4908, Loss: 0.08449279516935349,Ones#: 0.0008652735850773752, Accuracy: 95.35108947753906, Test Loss: 3.80499267578125,Test Ones#: 0.0009291726164519787, Test Accuracy: 61.6650505065918
Epoch 4909, Loss: 0.07955630123615265,Ones#: 0.0008621421875432134, Accuracy: 95.3995132446289, Test Loss: 3.7566936016082764,Test Ones#: 0.0009301311220042408, Test Accuracy: 62.19748306274414
Epoch 4910, Loss: 0.0788748636841774,Ones#: 0.0008643371984362602, Accuracy: 95.48426055908203, Test Loss: 3.7739737033843994,Test Ones#: 0.0009315653587691486, Test Accuracy: 62.100677490234375
Epoch 4911, Loss: 0.075688175857

Epoch 4949, Loss: 0.07195570319890976,Ones#: 0.0008572036167606711, Accuracy: 95.52057647705078, Test Loss: 3.8372182846069336,Test Ones#: 0.0009242596570402384, Test Accuracy: 62.826717376708984
Epoch 4950, Loss: 0.07292375713586807,Ones#: 0.0008609256474301219, Accuracy: 95.47216033935547, Test Loss: 3.8369603157043457,Test Ones#: 0.000926147389691323, Test Accuracy: 62.8751220703125
Epoch 4951, Loss: 0.07301148772239685,Ones#: 0.000860599335283041, Accuracy: 95.58111572265625, Test Loss: 3.850128650665283,Test Ones#: 0.0009247600100934505, Test Accuracy: 62.19748306274414
Epoch 4952, Loss: 0.07958170771598816,Ones#: 0.0008602261659689248, Accuracy: 95.4237289428711, Test Loss: 3.8069489002227783,Test Ones#: 0.0009262301027774811, Test Accuracy: 63.26234436035156
Epoch 4953, Loss: 0.08522455394268036,Ones#: 0.0008598091662861407, Accuracy: 95.04843139648438, Test Loss: 3.9087350368499756,Test Ones#: 0.0009271620656363666, Test Accuracy: 62.63311004638672
Epoch 4954, Loss: 0.090806707

Epoch 4992, Loss: 0.07458122074604034,Ones#: 0.0008645288180559874, Accuracy: 95.47216033935547, Test Loss: 3.8214120864868164,Test Ones#: 0.0009356898372061551, Test Accuracy: 62.826717376708984
Epoch 4993, Loss: 0.0740288719534874,Ones#: 0.0008652369724586606, Accuracy: 95.23002624511719, Test Loss: 3.867779016494751,Test Ones#: 0.0009350472246296704, Test Accuracy: 62.39109802246094
Epoch 4994, Loss: 0.0775788351893425,Ones#: 0.0008641078020446002, Accuracy: 95.32687377929688, Test Loss: 3.8273229598999023,Test Ones#: 0.0009334608912467957, Test Accuracy: 62.19748306274414
Epoch 4995, Loss: 0.07538123428821564,Ones#: 0.0008620161097496748, Accuracy: 95.38741302490234, Test Loss: 3.846721887588501,Test Ones#: 0.0009340335964225233, Test Accuracy: 62.149078369140625
Epoch 4996, Loss: 0.07337707281112671,Ones#: 0.0008640139130875468, Accuracy: 95.55690002441406, Test Loss: 3.86151385307312,Test Ones#: 0.0009358944953419268, Test Accuracy: 62.100677490234375
Epoch 4997, Loss: 0.07333132

* 在某些參數的情況下0 1是train不起來的但是weight可以
    * alpha = 0.0, beta=0.6
    * init_w = tensorflow.keras.initializers.Constant(value=0.9), init_b = tensorflow.keras.initializers.Constant(value=0.7)
* 0/1 with emb比較容易overfit。如果是weight的比較沒那麼嚴重

* 不用sigmoid或是hard_sigmoid。改良relu + linear，並拆成兩個model，把round前面多加上clip
    * sigmoid中間的變化太快(一瞬間就會掉到0或是1)，改成relu在>0~無限大(linear為了還在0~1)再去clip再round，可以看到每個epoch的變化
* 1st phase的beta一定要>=0.6否則不會動，ones#都會是0
    * beta=1 (放0 1進去): weight init設成1也沒用，但把bias設成1就會一開始都是ones#=1了。weight=1 bias=0.2都匯市0，bias=0.3會是0.87。0.6/0.3都是0。0.8/0.3差不多是0.5(但是train很慢acc進步很慢就是了)。0.5/0.5是從0.01開始往上升 (0 1放進去會比較難train是因為它的變化量太大，一下就是有或沒有，所以clf可能學不好，但如果是weight每次gradient進步的都是一小點就會比較容易上升)
    * beta=0.6 (放0 1進去): 0.8/0.5都是0。0.9/0.8 從0.9一直到0。0.9/0.6差不多是從0.5但又有時候到0.7都是0.0(很難train，Nadam換個opt有時候沒用。EX變成adam 0.9/0.8才有0.96開始但如果0.9/0.75變成0開始。Rmsprop 0.9/0.8又是從0.95開始往下)。但如果都改成傳入weight就都沒問題。ones一開始大概0.5 weight平均，也不會卡住
* 建議: 先訓練embedding weight matrix，但是要看goal是要怎樣的matrix
* 其實他不管幾%都會train得很好，除非固定embedding，或是用更弱的clf
* 若設定alpha，就像是regularizer term (penalty)，設越大drop越多
* 一開始先很多ones，再越來越少個

* 同一個opt若加入transform就會train不起來
* 兩個不同的opt加入transform也會train不起來 (persistent、non-persis都不行)，且與BN無關